# **Install and import nescessory package**

In [1]:
!pip install transformers==3.5.0

     |████████████████████████████████| 1.3MB 5.9MB/s 
     |████████████████████████████████| 1.1MB 36.5MB/s 
     |████████████████████████████████| 890kB 41.5MB/s 
     |████████████████████████████████| 2.9MB 44.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d69743eb405b51e1f36851e71dd9f6433a20550569193d86967c9a6a71d9d6d2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import MobileBertTokenizer, MobileBertModel, MobileBertForPreTraining
import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
device

'cuda'

# **try to figure out the structure**

In [ ]:
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased', truncation=True, do_lower_case=True )
inputs = tokenizer.encode_plus(
            'I love lavender I love lavender I love lavender I love lavender',
            None,
            add_special_tokens=True,
            max_length=10,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
ids = inputs['input_ids']
mask = inputs['attention_mask']
token_type_ids = inputs["token_type_ids"]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
ids

[101, 1045, 2293, 20920, 1045, 2293, 20920, 1045, 2293, 102]

In [ ]:
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
model = MobileBertModel.from_pretrained('google/mobilebert-uncased')
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

In [ ]:
inputs

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
outputs[0]

tensor([[[-2.5655e+07,  9.8468e+04,  1.6557e+05,  ..., -1.6260e+06,
           1.2349e+06,  2.6710e+04],
         [ 1.9210e-01,  6.7220e-01, -8.1362e-01,  ...,  5.5030e-02,
           1.4415e+00,  4.5810e+00],
         [ 9.1282e-01,  1.9443e+00,  1.5657e+00,  ..., -1.2405e-01,
          -2.7288e+00,  2.7489e+00],
         ...,
         [ 1.5894e+00,  5.9103e-01,  1.9070e+00,  ...,  2.7961e+00,
          -2.6210e+00,  3.7704e+00],
         [ 1.5775e+00,  3.8555e+00, -6.2034e-01,  ...,  2.9500e+00,
          -2.2804e+00,  2.9576e+00],
         [ 9.0074e-01,  8.5517e-01,  1.1304e+00,  ...,  1.1470e+00,
          -1.2494e+00,  6.7297e-01]]], grad_fn=<AddBackward0>)

we can see the output tensors shape is (512, )

# **Load training dataset**

In [4]:
data = pd.read_csv('/content/drive/My Drive/goemotions_aug_dairai_train_cleaned.csv')
data.drop(['Unnamed: 0', 'datasource', 'augmented'], inplace=True, axis=1)
new_df = pd.DataFrame()
new_df['text'] = data['cleaned_processed']
new_df['labels'] = data[['anger', 'fear', 'joy', 'sadness']].values.astype('int').tolist()

In [5]:
new_df.head()

,text,labels
0,i feel that some korea guy are handsome and so...,"[0, 0, 1, 0]"
1,i put my pen to paper and made a list of thing...,"[0, 0, 0, 0]"
2,i wish i only had to feel the pain of the pett...,"[1, 0, 0, 0]"
3,i feel passionate about this journey and stand...,"[0, 0, 1, 0]"
4,i feel like i have convinced myself of these f...,"[0, 0, 1, 0]"


In [6]:
new_df['len'] = new_df['text'].apply(lambda x: len(x.split(' ')))

In [7]:
new_df['len'].sort_values()

308960      1
198631      1
304858      1
164110      1
219996      1
         ... 
110501     80
224855     82
225137    101
78205     102
78768     180
Name: len, Length: 460551, dtype: int64

In [8]:
demo_df = new_df.sample(n=100000, random_state=42)

In [9]:
demo_df.head()

,text,labels,len
340232,but thank you anyway,"[0, 0, 1, 0]",4
136369,i could still feel the effect of the maple nut...,"[0, 0, 1, 0]",18
41752,i feel energetic and calm,"[0, 0, 1, 0]",5
162511,i feel more trusting of people,"[0, 0, 0, 0]",6
281481,i would not think about him sigh kinda make me...,"[0, 0, 0, 1]",26


In [10]:
MAX_LEN = 100
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
EPOCHS = 1
LEARNING_RATE = 3e-05

In [11]:

tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased', truncation=True, do_lower_case=True )

In [12]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

# **make samll dataloader to test the model**

In [ ]:
train_size = 0.7
train_data=demo_df.sample(frac=train_size,random_state=42)
test_data=demo_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(demo_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (100000, 3)
TRAIN Dataset: (70000, 3)
TEST Dataset: (30000, 3)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# **define MobileBertClass to finetune**

In [13]:
class MobileBERTClass(torch.nn.Module):
    def __init__(self):
        super(MobileBERTClass, self).__init__()
        self.l1 = MobileBertModel.from_pretrained("google/mobilebert-uncased")
        # self.pre_classifier = torch.nn.Linear(512, 512)
        self.dropout = torch.nn.Dropout(0.5)
        self.classifier = torch.nn.Linear(512, 4)
    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        # pooler = self.pre_classifier(pooler)
        # pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = MobileBERTClass()
model.to(device)

MobileBERTClass(
  (l1): MobileBertModel(
    (embeddings): MobileBertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (embedding_transformation): Linear(in_features=384, out_features=512, bias=True)
      (LayerNorm): NoNorm()
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): MobileBertEncoder(
      (layer): ModuleList(
        (0): MobileBertLayer(
          (attention): MobileBertAttention(
            (self): MobileBertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=512, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MobileBertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=T

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
# optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
optimizer = torch.optim.Adam(params = model.parameters(), lr=1e-04)

# **define training precess**

In [ ]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%50==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)
    

0it [00:00, ?it/s]

Epoch: 0, Loss:  0.17794927954673767


50it [00:18,  2.70it/s]

Epoch: 0, Loss:  0.11586861312389374


100it [00:37,  2.70it/s]

...

2100it [12:52,  2.73it/s]

Epoch: 0, Loss:  0.209250807762146


2150it [13:10,  2.73it/s]

Epoch: 0, Loss:  0.17611758410930634


2188it [13:24,  2.72it/s]


In [ ]:
output_model_file = '/content/drive/My Drive/Mobilebert_demo_emotions_11_11_epoch6.bin'
torch.save(model, output_model_file)

In [ ]:
!nvidia-smi

Wed Nov 11 21:32:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    33W /  70W |   4571MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Validation**

In [ ]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [ ]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

8636it [07:42, 18.69it/s]


In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

Hamming Score = 0.8419570803025368
Hamming Loss = 0.041993268917598525


**evalution I think it fails, an don't know why**


1.   /
2.   epoch 2: 
          Hamming Score = 0.34155
          Hamming Loss = 0.178075
3.   epoch 3:
          Hamming Score = 0.3859166666666667
          Hamming Loss = 0.17255833333333334
4.   epoch 4:
          Hamming Score = 0.4126166666666667
          Hamming Loss = 0.16695
5.   epoch 5 (using larger learning rate 1e-4):
          Hamming Score = 0.8067277777777778
          Hamming Loss = 0.05218333333333333
6.   epoch 5 (using larger learning rate 1e-4):
          Hamming Score = 0.8290722222222221
          Hamming Loss = 0.045875

I think we are ready to train on the whole dataset

# **train on the whole data set**

In [14]:
train_size = 0.7
train_data=new_df.sample(frac=train_size,random_state=42)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (460551, 3)
TRAIN Dataset: (322386, 3)
TEST Dataset: (138165, 3)


In [15]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [16]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%200==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)


0it [00:00, ?it/s]

Epoch: 0, Loss:  0.1490464210510254


200it [01:13,  2.68it/s]

Epoch: 0, Loss:  0.06480734795331955


400it [02:26,  2.75it/s]

...

9800it [58:32,  2.78it/s]

Epoch: 0, Loss:  0.04704207926988602


10000it [59:44,  2.78it/s]

Epoch: 0, Loss:  0.08049255609512329


10075it [1:00:11,  2.79it/s]


In [ ]:
output_model_file = '/content/drive/My Drive/Mobilebert_demo_emotions_11_11_whole_dataset_epoch1.bin'
torch.save(model, output_model_file)

*   First epoch
        Hamming Score = 0.8419570803025368
        Hamming Loss = 0.041993268917598525

# **Evaluation on isear dataset**

In [ ]:
df = pd.read_csv('/content/drive/My Drive/isear_processed_emotions.csv')
df = pd.get_dummies(df, columns=['emotions'])
df['labels'] = df[['emotions_anger', 'emotions_fear', 'emotions_joy', 'emotions_sadness']].values.astype('int').tolist()
df = df[(df['emotions_anger']==1) | (df['emotions_fear']==1) | (df['emotions_joy']==1) | (df['emotions_sadness']==1)]


In [ ]:
!pip install contractions

     |████████████████████████████████| 317kB 15.7MB/s 
     |████████████████████████████████| 245kB 28.1MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81689 sha256=0c7baa24612f3a4c71d59757c2ed3c2a2052ae25556a231ac2f682c9aea03d02
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
import re
import contractions
import unicodedata
import string

In [ ]:
def clean_text(text):
    """
    Function to clean text with basic steps - lower casing, dealing with contractions, remove html codes,
    strip whitespaces, social media cleaning (remove hashtags and URLS), remove punctuationns, using regular expressions.
 
    Parameters
    ----------
    text : str
            Text to be cleaned
    
    Returns
    -------
    text : str
            Cleaned text
    """
    # Lower casing
    text = text.lower()
    
    
    # Remove html codes
    text = re.sub(r"&amp;", " ", text)
    text = re.sub(r"&quot;", " ", text)
    text = re.sub(r"&#39;", " ", text)
    text = re.sub(r"&gt;", " ", text)
    text = re.sub(r"&lt;", " ", text)
    
    # Strips (removes) whitespaces
    text = text.strip(' ')
    
    ################ Social media cleaning ############
    
    # Remove hashtags (Regex @[A-Za-z0-9]+ represents mentions and #[A-Za-z0-9]+ represents hashtags. )
    text = re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", text)
    
    # Remove URLS (Regex \w+:\/\/\S+ matches all the URLs starting with http:// or https:// and replacing it with space.)
    text = re.sub("(\w+:\/\/\S+)", " ", text)
    text = re.sub(r'http\S+', ' ', text)
    
     # remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # remove accents
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # remove @users
    text = re.sub(r'@[\w]*', '', text)
    # remove Reddit channel reference /r
    text = re.sub(r'r/', '', text)
    
    # remove reddit username
    text = re.sub(r'u/[\w]*', '', text)
    # remove '&gt;' like notations
    text = re.sub('&\W*\w*\W*;', ' ', text)
    # remove hashtags
    text = re.sub(r'#[\w]*', '', text)
    ###################################################
    
    # Dealing with contractions
    text = contractions.fix(text)
    
    text = re.sub(r"what\'s", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can\'t", "can not ", text)
    text = re.sub(r"n\'t", " not ", text)
    text = re.sub(r"\'t", " not", text )
    text = re.sub(r"i\'m", "i am ", text)
    text = re.sub(r"\'em'", " them ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    
    
    # Removes punctuations
    text = re.sub('['+string.punctuation+']', " ", text)
    
	# Removes non alphanumeric characters
    #text = re.sub('\W', ' ', text)
    
    # Removes non alphabetical characters
    text = re.sub('[^a-zA-Z]+', ' ', text)
    
    # Replaces all whitespaces by 1 whitespace
    text = re.sub('\s+', ' ', text)
    
    return text

In [ ]:
df['text'] = df['text'].apply(clean_text)

In [ ]:
model.eval()
pred = []
from tqdm import tqdm
for text in tqdm(df.text.values):
  input = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=MAX_LEN,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
  ids = torch.tensor([input['input_ids']], dtype=torch.long)
  mask = torch.tensor([input['attention_mask']], dtype=torch.long)
  token_type_ids = torch.tensor([input["token_type_ids"]], dtype=torch.long)
  # to device
  ids = ids.to(device, dtype = torch.long)
  mask = mask.to(device, dtype = torch.long)
  token_type_ids = token_type_ids.to(device, dtype = torch.long)

  output = model(ids, mask, token_type_ids)
  pred.append(torch.sigmoid(output).cpu().detach().numpy().tolist())


100%|██████████| 4381/4381 [04:27<00:00, 16.39it/s]


In [ ]:
pred_y = np.array(pred)
pred_y = pred_y.squeeze(axis=1)
for i in range(len(pred_y)):
  for j in range(4):
    pred_y[i, j] =1 if pred_y[i, j] >= 0.5 else 0
pred_y = pred_y.astype('int')

real_y = df.labels.values.tolist()
real_y = np.array(real_y)
real_y = real_y.astype('int')

**overall metrics**

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
precision_score(real_y, pred_y, average='macro')

0.7949025458427011

In [ ]:
recall_score(real_y, pred_y, average='macro')

0.5714911446088786

In [ ]:
f1_score(real_y, pred_y, average='macro')

0.601545378492069

**precision score**

In [ ]:
precision = [precision_score(real_y[:, 0], pred_y[:, 0]),
             precision_score(real_y[:, 1], pred_y[:, 1]),
             precision_score(real_y[:, 2], pred_y[:, 2]),
             precision_score(real_y[:, 3], pred_y[:, 3]),
             precision_score(real_y, pred_y, average='macro'),
             precision_score(real_y, pred_y, average='micro'),
             precision_score(real_y, pred_y, average='weighted')]

**recall score**

In [ ]:
recall = [recall_score(real_y[:, 0], pred_y[:, 0]),
          recall_score(real_y[:, 1], pred_y[:, 1]),
          recall_score(real_y[:, 2], pred_y[:, 2]),
          recall_score(real_y[:, 3], pred_y[:, 3]),
          recall_score(real_y, pred_y, average='macro'),
          recall_score(real_y, pred_y, average='micro'),
          recall_score(real_y, pred_y, average='weighted')]

**f1 score**

In [ ]:
f1 = [f1_score(real_y[:, 0], pred_y[:, 0]),
      f1_score(real_y[:, 1], pred_y[:, 1]),
      f1_score(real_y[:, 2], pred_y[:, 2]),
      f1_score(real_y[:, 3], pred_y[:, 3]),
      f1_score(real_y, pred_y, average='macro'),
      f1_score(real_y, pred_y, average='micro'),
      f1_score(real_y, pred_y, average='weighted')]

**overall**

In [ ]:
metric = pd.DataFrame([precision, recall, f1], columns=['anger', 'fear', 'joy', 'sadness', 'macro', 'micro', 'weighted'], index=['precision', 'recall','f1'])

In [ ]:
metric

,anger,fear,joy,sadness,macro,micro,weighted
precision,0.443649,0.884309,0.922917,0.928736,0.794903,0.641558,0.794824
recall,0.905109,0.607306,0.404936,0.368613,0.571491,0.571559,0.571559
f1,0.595438,0.720087,0.562897,0.527760,0.601545,0.604539,0.601536


# **load the model and train on the whole data set fro some more epochs**

In [17]:
train_data=new_df
training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
training_loader = DataLoader(training_set, **train_params)

the load time is longer than before...

In [18]:
class MobileBERTClass(torch.nn.Module):
    def __init__(self):
        super(MobileBERTClass, self).__init__()
        self.l1 = MobileBertModel.from_pretrained("google/mobilebert-uncased")
        self.dropout = torch.nn.Dropout(0.5)
        self.classifier = torch.nn.Linear(512, 4)
    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = MobileBERTClass()

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/mobileBert_demo_emotions_state_dict_11_14_0'), strict=False)

In [ ]:
model.to(device)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%200==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

after the pretraining, we should use smaller learning rate to train on the training set.

in the same training stage, the performance is much worse than distilBert, which is not expectable

and also is oscillating.... it seems it doesn't converging

In [ ]:
EPOCHS = 3
for epoch in range(EPOCHS):
    train(epoch)
    torch.save(model.state_dict(), f'/content/drive/My Drive/mobileBert_demo_emotions_state_dict_11_14_{epoch}')

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch: 0, Loss:  0.07767188549041748


200it [01:07,  3.02it/s]

Epoch: 0, Loss:  0.0635480061173439


400it [02:13,  3.07it/s]

...

3200it [17:19,  2.83it/s]

Epoch: 1, Loss:  0.05773472413420677


3400it [18:24,  3.20it/s]

Epoch: 1, Loss:  0.03903431445360184


3422it [18:32,  3.01it/s]